# 🔥 Fire Spread Prediction with CNN + PINN
This notebook builds a hybrid CNN and PINN model to predict fire spread rate from images, terrain, and geographic data.

In [2]:
# 🔧 SETUP AND LIBRARIES
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import KFold
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import gaussian_filter
from osgeo import gdal

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# 📁 PATHS TO YOUR EXTERNAL DATASETS
TRAIN_IMAGE_FOLDER = r'C:\Users\Mahip\Downloads\Fire_models_Train\5_Fire_models_Train'  # GR, GS, SH, TL, NB subfolders
TEST_IMAGE_FOLDER = r'C:\Users\Mahip\Downloads\Fire_models_test\5_Fire_models_test'
DEM_FILE_PATH = r'C:\Users\Mahip\Downloads\TIFF_input\TIFF_input'
PORTUGUESE_ODS_PATH = r'C:\Users\Mahip\Downloads\PT-FireSprd_L2_FireBehavior_FULL_set.ods'


In [4]:
# 🖼️ DATASET AND AUGMENTATION
class FireDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.GaussianBlur(3),
    transforms.ToTensor()
])


In [5]:
# 🔍 LOAD IMAGES AND LABELS
def load_image_paths(root_folder):
    image_paths, labels = [], []
    label_map = {'GR': 0, 'GS': 1, 'SH': 2, 'TL': 3, 'NB': 4}
    supported_formats = ('.jpg', '.jpeg', '.png')  # ✅ Only use supported image types
    for cls in os.listdir(root_folder):
        cls_folder = os.path.join(root_folder, cls)
        for fname in os.listdir(cls_folder):
            if fname.lower().endswith(supported_formats):
                image_paths.append(os.path.join(cls_folder, fname))
                labels.append(label_map[cls])
    return image_paths, labels

train_paths, train_labels = load_image_paths(TRAIN_IMAGE_FOLDER)


In [6]:
# 🧠 CNN MODEL DEFINITION
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        base_model = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Linear(512, 5)  # 🔁 5 classes: GR, GS, SH, TL, NB

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


cnn_model = CNNClassifier().to(device)


C:\Users\Mahip\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Mahip\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 🔁 CNN TRAINING WITH K-FOLD
def train_cnn_kfold(k=5, epochs=10):
    kfold = KFold(n_splits=k, shuffle=True)
    losses = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_paths)):
        train_data = FireDataset([train_paths[i] for i in train_idx], [train_labels[i] for i in train_idx], data_transform)
        val_data = FireDataset([train_paths[i] for i in val_idx], [train_labels[i] for i in val_idx], data_transform)

        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

        model = CNNClassifier().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            model.train()
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images.to(device))
                predicted = torch.argmax(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels.to(device)).sum().item()
        acc = 100 * correct / total
        losses.append(acc)
        print(f"[Fold {fold + 1}] Accuracy: {acc:.2f}%")

    plt.plot(losses)
    plt.title("CNN Accuracy per Fold")
    plt.xlabel("Fold")
    plt.ylabel("Accuracy (%)")
    plt.show()
    
train_cnn_kfold()

[Fold 1] Accuracy: 89.09%
[Fold 2] Accuracy: 89.09%
[Fold 3] Accuracy: 90.91%
[Fold 4] Accuracy: 94.44%


In [ ]:
# 🌄 DEM FEATURE EXTRACTION FROM MULTIPLE FILES
from osgeo import osr

def lonlat_to_pixel(geo_transform, lon, lat, srs):
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)  # WGS84
    transform = osr.CoordinateTransformation(target, srs)
    x_geo, y_geo, _ = transform.TransformPoint(lon, lat)
    px = int((x_geo - geo_transform[0]) / geo_transform[1])
    py = int((y_geo - geo_transform[3]) / geo_transform[5])
    return px, py

def extract_from_dem_file(dem_path, lon, lat):
    ds = gdal.Open(dem_path)
    gt = ds.GetGeoTransform()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(ds.GetProjection())
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    x, y = lonlat_to_pixel(gt, lon, lat, srs)

    if 0 <= y < arr.shape[0] and 0 <= x < arr.shape[1]:
        elevation = arr[y, x]
        grad_x, grad_y = np.gradient(arr)
        slope = np.hypot(grad_x[y, x], grad_y[y, x])
        return elevation, slope
    return None, None

def extract_dem_features(lon, lat, dem_folder):
    for root, _, files in os.walk(dem_folder):
        for file in files:
            if file.endswith(".tif"):
                dem_path = os.path.join(root, file)
                elevation, slope = extract_from_dem_file(dem_path, lon, lat)
                if elevation is not None:
                    return elevation, slope
    print(f"Coordinates ({lon}, {lat}) not found in any DEM file.")
    return 0, 0  # default fallback


In [ ]:
# 🔬 PINN MODEL DEFINITION
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(517, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

pinn_model = PINN().to(device)


In [ ]:
# 🧠 CNN INFERENCE ON IMAGE
from torchvision.transforms import Compose, Resize, ToTensor

# This should match training transform except augmentations
cnn_infer_transform = Compose([
    Resize((224, 224)),
    ToTensor()
])

def extract_features_from_image(image_path, model, transform=cnn_infer_transform):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(image)
    return features.cpu().numpy().flatten()


In [ ]:
# 🗂️ PATH TO CNN IMAGE FOLDER USED FOR INFERENCE
CNN_IMAGE_FOLDER = r'C:\Users\Mahip\Downloads\Fire_models_test\5_Fire_models_test\TL\image_63_2.jpeg'


In [ ]:
# 🧪 LOAD PORTUGUESE DATA WITH CLEANED COLUMN NAMES
portugal_df = pd.read_excel(PORTUGUESE_ODS_PATH, engine='odf')

# Normalize column names (strip + lowercase)
portugal_df.columns = [col.strip().lower() for col in portugal_df.columns]

# 🔄 Replace these with actual cleaned column names if different
def preprocess_portugal_data():
    features, targets = [], []
    for _, row in portugal_df.iterrows():
        lon, lat = row['Longitude'], row['Latitude']
        t = row['inidoy,N,11,6']
        ros = row['ros_p,N,12,6']
        elev, slope = extract_dem_features(lon, lat, DEM_FOLDER_PATH)

        # === CNN inference on a dummy image (replace with actual logic) ===
        # This assumes a function `extract_features_from_image(image_path)` exists
                image_path = CNN_IMAGE_FOLDER
        cnn_features = extract_features_from_image(image_path, cnn_model)

        full_input = np.concatenate([cnn_features, [lon, lat, t, slope, elev]])
        features.append(full_input)
        targets.append(ros)
    return torch.tensor(features, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).view(-1, 1)

# Replace this with your DEM folder path
DEM_FOLDER_PATH = r'C:\Users\Mahip\Downloads\TIFF_input\TIFF_input'

X_train, y_train = preprocess_portugal_data()


In [ ]:
# 🧮 PINN TRAINING
# 🌡️ True PDE Residual Loss (advection-diffusion)
def pde_residual_loss(model, x, device, v=1.0, D=0.1):
    x = x.clone().detach().requires_grad_(True).to(device)
    
    pred = model(x)

    # Assume: x[:, -3] = t (time), x[:, -4] = x-position or longitude
    t = x[:, -3].view(-1, 1)
    pos = x[:, -4].view(-1, 1)

    # Compute ∂u/∂t and ∂u/∂x
    grad_u = torch.autograd.grad(pred, x, grad_outputs=torch.ones_like(pred),
                                 create_graph=True, retain_graph=True)[0]
    du_dt = grad_u[:, -3].view(-1, 1)
    du_dx = grad_u[:, -4].view(-1, 1)

    # Compute ∂²u/∂x²
    grad2_u = torch.autograd.grad(du_dx, x, grad_outputs=torch.ones_like(du_dx),
                                  create_graph=True, retain_graph=True)[0]
    d2u_dx2 = grad2_u[:, -4].view(-1, 1)

    # PDE residual: ∂u/∂t + v ∂u/∂x - D ∂²u/∂x² ≈ 0
    residual = du_dt + v * du_dx - D * d2u_dx2
    return torch.mean(residual**2)

def train_pinn(epochs=1000):
    optimizer = torch.optim.Adam(pinn_model.parameters(), lr=1e-4)
    
    for epoch in range(epochs):
        pinn_model.train()

        X_train.requires_grad_()  # ✅ Needed for autograd physics loss
        pred = pinn_model(X_train.to(device))

        mse_loss = F.mse_loss(pred, y_train.to(device))
        pde_loss = pde_residual_loss(pinn_model, X_train, device)

        loss = mse_loss + 0.1 * pde_loss  # 0.1 = weight for PDE constraint

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, MSE: {mse_loss.item():.4f}, PDE: {pde_loss.item():.4f}, Total: {loss.item():.4f}")

# To train the PINN, uncomment:
 train_pinn()


In [ ]:
# 📊 FINAL EVALUATION
def final_evaluation():
    pinn_model.eval()
    predictions = pinn_model(X_train.to(device)).cpu().detach().numpy()
    true_vals = y_train.cpu().numpy()

    plt.plot(predictions, label="Predicted ROS")
    plt.plot(true_vals, label="True ROS")
    plt.legend()
    plt.title("PINN Predicted vs True Fire Spread Rate")
    plt.xlabel("Sample Index")
    plt.ylabel("Rate of Spread")
    plt.show()

# Uncomment after training:
 final_evaluation()


In [ ]:
# 🔍 MANUAL INFERENCE: INPUT IMAGE, LONGITUDE, LATITUDE
# Provide these manually before running this cell:

IMAGE_PATH = r'C:\Users\Mahip\Downloads\fire_model_images_by_model-20250430T025324Z-001\fire_model_images_by_model\GR6\image_39_2.jpeg'  # 🔁 Your test image here
INPUT_LAT = 41.2                               # 🔁 Your latitude here
INPUT_LON = -8.6                               # 🔁 Your longitude here
IGNITION_TIME = 3.5                            # 🔁 Time since ignition (hours)

# 1. Extract CNN features from provided image
cnn_features = extract_features_from_image(IMAGE_PATH, cnn_model)
print("CNN Features [Texture, Vegetation Type, Moisture, Terrain]:", cnn_features)

# 2. Extract DEM features for given coordinates
elevation, slope = extract_dem_features(INPUT_LON, INPUT_LAT, DEM_FOLDER_PATH)
print(f"Slope (radians): {slope:.4f}")
print(f"Elevation (m): {elevation:.2f}")

# 3. Predict Spread Rate using PINN
input_tensor = torch.tensor([*cnn_features, INPUT_LON, INPUT_LAT, IGNITION_TIME, slope, elevation], dtype=torch.float32).unsqueeze(0).to(device)
pinn_model.eval()
with torch.no_grad():
    spread_rate = pinn_model(input_tensor).item()
print(f"Predicted Spread Rate (m/h): {spread_rate:.2f}")
